# How to painlessly transform an NLP model in Jupyter to a production API?

# [SMS Spam Collection Data Set](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection)

<img src="https://archive.ics.uci.edu/ml/assets/logo.gif" align='left' />

# Bag-of-words model with word embeddings learned from scratch

<a href="https://colab.research.google.com/github/Paulescu/practical-nlp-2021/blob/main/spam_detection/noteboooks/model.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" align="left"/>
</a>

### Required setup if you run the notebook in Google Colab

In [1]:
# # you need to paste the URL of your Github repo here if you want to run this notebook in Google Colab.
# URL_GITHUB_REPO = 'https://github.com/Paulescu/practical-nlp-2021'

# # a hacky way to check if the current notebook is running in Google Colab.
# if 'google.colab' in str(get_ipython()):
#     # we are running notebook in Colab
#     !git clone $URL_GITHUB_REPO
#     !cd .. && python setup.py develop
# else:
#     print('Python setup skiped.')

# Step 1. Download data and split into train, validation and test

The dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection)

### Download raw data

In [90]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
!tar -xf smsspamcollection.zip

--2020-12-14 18:39:15--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203415 (199K) [application/x-httpd-php]
Saving to: ‘smsspamcollection.zip.2’

smsspamcollection.z 100%[===================>] 198.65K   269KB/s    in 0.7s    

2020-12-14 18:39:17 (269 KB/s) - ‘smsspamcollection.zip.2’ saved [203415/203415]



### Quick data exploration

In [2]:
import pandas as pd

data = pd.read_csv('SMSSpamCollection', sep='\t', header=None)
data.columns = ['label', 'text']

In [3]:
data.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
data['label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: label, dtype: float64

### Add numeric column for the label

In [5]:
IDX_TO_LABEL = {
    0: 'ham',
    1: 'spam',
}

LABEL_TO_IDX = {
    'ham': 0,
    'spam': 1,
}

data['label_int'] = data['label'].apply(lambda x: LABEL_TO_IDX[x])
data.head()

,label,text,label_int
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


### Split data into files `train.csv` , `validation.csv`, `test.csv`

In [6]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.20, random_state=123,)
train_data, validation_data = train_test_split(train_data, test_size=0.20, random_state=123)

print('train_data: ', len(train_data))
print('validation_data: ', len(validation_data))
print('test_data: ', len(test_data))

train_data[['label_int', 'text']].to_csv('train.csv', index=False, header=False)
validation_data[['label_int', 'text']].to_csv('validation.csv', index=False, header=False)
test_data[['label_int', 'text']].to_csv('test.csv', index=False, header=False)

train_data:  3565
validation_data:  892
test_data:  1115


# Step 2. Define PyTorch `DataLoader`s for train, validation, and test.

In [1]:
import pandas as pd

# train_texts, train_labels
train_data = pd.read_csv('train.csv', header=None)
train_data.columns = ['label', 'text']
train_texts = train_data['text'].tolist()
train_labels = train_data['label'].tolist()

# validation_texts, validation_labels
validation_data = pd.read_csv('validation.csv', header=None)
validation_data.columns = ['label', 'text']
validation_texts = validation_data['text'].tolist()
validation_labels = validation_data['label'].tolist()

# test_texts, test_labels
test_data = pd.read_csv('test.csv', header=None)
test_data.columns = ['label', 'text']
test_texts = test_data['text'].tolist()
test_labels = test_data['label'].tolist()

In [2]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
validation_encodings = tokenizer(validation_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [3]:
import torch

class SpamDetectionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SpamDetectionDataset(train_encodings, train_labels)
validation_dataset = SpamDetectionDataset(validation_encodings, validation_labels)
test_dataset = SpamDetectionDataset(test_encodings, test_labels)

In [8]:
# Setup logging to Tensorboard
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

now = datetime.now()
now = now.strftime("%Y-%m-%d-%H:%M:%S")
MODEL_NAME = 'fine_tuning_bert'
log_file = f'./runs/{MODEL_NAME}/{now}'
writer = SummaryWriter(log_file)

# Train lopp
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.to(device)

# data loaders
BATCH_SIZE = 16
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)

# optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

N_EPOCHS = 150
for epoch in range(N_EPOCHS):
    
    # train
    running_loss = 0.0
    model.train()
    train_size = 0
    running_accuracy = 0.0
    
    for batch in tqdm(train_loader):
        
        # forward pass to compute the batch loss       
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        predictions = outputs[1]
        
        # backward pass to update model parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # compute train metrics
        running_loss += loss.data * input_ids.size(0)
        _, predicted_classes = torch.max(predictions, 1)
        running_accuracy += predicted_classes.eq(labels.data).sum().item()
        train_size += input_ids.size(0)
        
    epoch_loss = running_loss / train_size
    epoch_accuracy = running_accuracy / train_size
    
    # validation
    val_loss = 0.0
    model.eval()
    val_size = 0
    val_accuracy = 0
    with torch.no_grad():
        for batch in validation_loader:
            
            # forward pass
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs[0]
            predictions = outputs[1]
            
            # compute validation metrics
            val_loss += loss.data * input_ids.size(0)
            _, predicted_classes = torch.max(predictions, 1)
            val_accuracy += predicted_classes.eq(labels.data).sum().item()           
            val_size += input_ids.size(0)
            
        val_loss /= val_size
        val_accuracy /= val_size
        
        print('\nEpoch: {}'.format(epoch))
        print('Loss \t Train: {:.4f} \t Validation: {:.4f}'.format(epoch_loss, val_loss))
        print('Acc: \t Train: {:.4f} \t Validation: {:.4f}'.format(epoch_accuracy, val_accuracy))

    # log metrics to tensorboard
    writer.add_scalars('Loss', {'train': epoch_loss, 'validation': val_loss}, epoch + 1)
    writer.add_scalars('Accuracy', {'train': epoch_accuracy, 'validation': val_accuracy}, epoch + 1)
    
writer.close()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

KeyboardInterrupt: 

In [ ]:

# for epoch in range(3):
#     for batch in train_loader:
#         optim.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        
#         loss = outputs[0]
#         loss.backward()
#         optim.step()

# model.eval()

#### Check output from `Dataloader`

In [98]:
train_input = next(iter(train_iter))

print(train_input.text)
print(train_input.label)

tensor([[  68,    2,   34,  ...,   39,  349,    2],
        [ 156,   21,    3,  ..., 1984,   37,  197],
        [   0,  178,   46,  ...,  179,   57,    8],
        ...,
        [ 110,   99,  959,  ...,  443,    8,    1],
        [1963,   73,    0,  ...,  691,   10,    1],
        [  63,  299,    9,  ...,   21,    8,    1]])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])


/Users/paulabartabajo/src/online-courses/practical-nlp-2021/spam_detection/.venv/lib/python3.7/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [99]:
for i in range(10):
    print('text: ', train[i].text)
    print('label: ', train[i].label)
    print('---')

text:  ['mom', 'wants', 'to', 'know', 'where', 'you', 'at']
label:  0
---
text:  ['boy', ';', 'i', 'love', 'u', 'grl', ':', 'hogolo', 'boy', ':', 'gold', 'chain', 'kodstini', 'grl', ':', 'agalla', 'boy', ':', 'necklace', 'madstini', 'grl', ':', 'agalla', 'boy', ':', 'hogli', '1', 'mutai', 'eerulli', 'kodthini', '!', 'grl', ':', 'i', 'love', 'u', 'kano;-', ')']
label:  0
---
text:  ['its', 'on', 'in', 'engalnd', '!', 'but', 'telly', 'has', 'decided', 'it', 'wo', "n't", 'let', 'me', 'watch', 'it', 'and', 'mia', 'and', 'elliot', 'were', 'kissing', '!', 'damn', 'it', '!']
label:  0
---
text:  ['your', 'gon', 'na', 'have', 'to', 'pick', 'up', 'a', '$', '1', 'burger', 'for', 'yourself', 'on', 'your', 'way', 'home', '.', 'i', 'ca', "n't", 'even', 'move', '.', 'pain', 'is', 'killing', 'me', '.']
label:  0
---
text:  ['no', 'no:)this', 'is', 'kallis', 'home', 'ground.amla', 'home', 'town', 'is', 'durban', ':', ')']
label:  0
---
text:  ['i', 'am', 'seeking', 'a', 'lady', 'in', 'the', 'street', 

# Step 3. Define the neural net model

In [100]:
# TODO: add diagram here

In [101]:
import torch.nn as nn
import torch.nn.functional as F

def global_max_pool(x):
    y, _ = torch.max(x, dim=-1)
    return y
    
class Model(nn.Module):
    
    def __init__(self, vocab_size: int, embedding_dim: int, n_filters: int):
        super(Model, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        
        self.bigram = nn.Conv1d(embedding_dim, n_filters, 2)
        self.trigram = nn.Conv1d(embedding_dim, n_filters, 3)       

        self.fc1 = nn.Linear(n_filters * 2, 16)
        self.dropout = nn.Dropout(0.25)
        self.fc2 = nn.Linear(16, 2)
        
    def forward(self, x):
#         print(x.shape)
        x = self.embed(x)
        x = x.transpose(-1, -2)
#         print(x.shape)
        
        # bigram branch
        x_1 = self.bigram(x)
        x_1 = F.relu(x_1)
#         print(x_1.shape)
        x_1 = global_max_pool(x_1)
#         print(x_1.shape)
        
        # trigram branch
        x_2 = self.trigram(x)
        x_2 = F.relu(x_2)
        x_2 = global_max_pool(x_2)
#         print(x_2.shape)
        
        x = torch.cat((x_1, x_2), 1)
#         print(x.shape)
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

EMBEDDING_DIM = 16

model = Model(vocab_size, EMBEDDING_DIM, n_filters=15).to(DEVICE)

# # debugging
# test_input = next(iter(train_iter)).text
# print('input: ', test_input.shape)
# test_output = model(test_input)
# print('output: ', test_output.shape)

# Step 4. Train the model

### Loss function and optimizer

In [102]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

### Launch Tensorboard

In [103]:
%load_ext tensorboard
%tensorboard --logdir runs

Reusing TensorBoard on port 6008 (pid 79472), started 1:33:04 ago. (Use '!kill 79472' to kill it.)

### Train loop

In [104]:
# Setup logging to Tensorboard
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

now = datetime.now()
now = now.strftime("%Y-%m-%d-%H:%M:%S")
MODEL_NAME = 'bag_of_words_embeddings_glove'
log_file = f'./runs/{MODEL_NAME}/{now}'
writer = SummaryWriter(log_file)

# Train lopp
from tqdm import tqdm
N_EPOCHS = 150
for epoch in range(N_EPOCHS):
    
    # train
    running_loss = 0.0
    model.train()
    train_size = 0
    running_accuracy = 0.0
    for batch in tqdm(train_iter):
        
        # forward pass to compute the batch loss
        x = batch.text
        y = batch.label.long()
        predictions = model(x)        
        loss = criterion(predictions, y)
            
        # backward pass to update model parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # compute train metrics
        running_loss += loss.data * x.size(0)
        _, predicted_classes = torch.max(predictions, 1)
        running_accuracy += predicted_classes.eq(y.data).sum().item()
        train_size += x.size(0)
        
    epoch_loss = running_loss / train_size
    epoch_accuracy = running_accuracy / train_size
    
    # validation
    val_loss = 0.0
    model.eval()
    val_size = 0
    val_accuracy = 0
    with torch.no_grad():
        for batch in validation_iter:
            x = batch.text
            y = batch.label.long()
            predictions = model(x)
            loss = criterion(predictions, y)
            
            # compute validation metrics
            val_loss += loss.data * x.size(0)
            _, predicted_classes = torch.max(predictions, 1)
            val_accuracy += predicted_classes.eq(y.data).sum().item()           
            val_size += x.size(0)
            
        val_loss /= val_size
        val_accuracy /= val_size
        
        print('\nEpoch: {}'.format(epoch))
        print('Loss \t Train: {:.4f} \t Validation: {:.4f}'.format(epoch_loss, val_loss))
        print('Acc: \t Train: {:.4f} \t Validation: {:.4f}'.format(epoch_accuracy, val_accuracy))

    # log metrics to tensorboard
    writer.add_scalars('Loss', {'train': epoch_loss, 'validation': val_loss}, epoch + 1)
    writer.add_scalars('Accuracy', {'train': epoch_accuracy, 'validation': val_accuracy}, epoch + 1)
    
writer.close()

 32%|███▏      | 9/28 [00:00<00:00, 78.78it/s]


Epoch: 0
Loss 	 Train: 0.6048 	 Validation: 0.5319
Acc: 	 Train: 0.8468 	 Validation: 0.8857


 39%|███▉      | 11/28 [00:00<00:00, 100.27it/s]


Epoch: 1
Loss 	 Train: 0.5021 	 Validation: 0.4456
Acc: 	 Train: 0.8620 	 Validation: 0.8857


 32%|███▏      | 9/28 [00:00<00:00, 87.13it/s]

...


Epoch: 147
Loss 	 Train: 0.0021 	 Validation: 0.0365
Acc: 	 Train: 0.9994 	 Validation: 0.9787


 29%|██▊       | 8/28 [00:00<00:00, 79.06it/s]


Epoch: 148
Loss 	 Train: 0.0021 	 Validation: 0.0360
Acc: 	 Train: 0.9997 	 Validation: 0.9787


100%|██████████| 28/28 [00:00<00:00, 75.06it/s]


Epoch: 149
Loss 	 Train: 0.0021 	 Validation: 0.0365
Acc: 	 Train: 0.9997 	 Validation: 0.9776


# Step 5. Test the model

In [105]:
test_accuracy = 0.0
test_size = 0
with torch.no_grad():
    for batch in test_iter:
        # forward pass
        x = batch.text
        y = batch.label.long()
        predictions = model(x)        
        loss = criterion(predictions, y)

        # compute accuracy
        _, predicted_classes = torch.max(predictions, 1)
        test_accuracy += predicted_classes.eq(y.data).sum().item()
        test_size += x.size(0)

test_accuracy /= test_size
print('Test accuracy: {:.4f}'.format(test_accuracy))

Test accuracy: 0.9785


# Extra. Interact with the model
Pay attention how pre-processing and post-processing are necessary to be able to use the model at inference time.

https://github.com/bentrevett/pytorch-sentiment-analysis/issues/40

In [19]:
sentences = [
    'This is your friend Carl. Come to the Casino and get a discount!',
    'This is your friend Carl, do you want to meet later?',
    'Would you be interested in buying a car for nothing?',
    'Send your card details today and get a prize!',
    'I won two tickets to the show, do you want to come with me?',
    'I won two tickets to the show, just send an SMS to this number and get them',
]

for s in sentences:
    # pre-process text into integer tokens
    model_input = [TEXT.vocab.stoi[token] for token in tokenizer_fn(s)]
    # add 0-dimension
    model_input = torch.LongTensor(model_input).unsqueeze(0)
    
    # run model prediction
    predictions = model(model_input)
    
    # post-processing
    _, predicted_class = torch.max(predictions, 1)
    predicted_class = predicted_class.item()
    predicted_class_str = IDX_TO_LABEL[predicted_class]
    
    # print
    print(s)
    print(predicted_class_str)
    print('------')

This is your friend Carl. Come to the Casino and get a discount!
spam
------
This is your friend Carl, do you want to meet later?
ham
------
Would you be interested in buying a car for nothing?
ham
------
Send your card details today and get a prize!
spam
------
I won two tickets to the show, do you want to come with me?
ham
------
I won two tickets to the show, just send an SMS to this number and get them
ham
------


# Extra: Visualize the learned word embeddings with the [Embedding Projector](https://projector.tensorflow.org/)

### Extract embedding parameters

In [ ]:
for name, parameter in model.named_parameters():
    if name == 'embed.weight':
        embeddings = parameter

print(embeddings.shape)

### Generate tsv files

In [ ]:
import io

embeddings = embeddings.cpu().detach().numpy()
vocab = TEXT.vocab.itos

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
    if index in [0, 1]:
        # skip 0, it's the unknown token.
        # skip 1, it's the padding token.
        continue
        
    vec = embeddings[index, :] 
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
    out_m.write(word + "\n")

out_v.close()
out_m.close()

### Download files to your local computer (in case you are running this notebook in Google Colab)

In [ ]:
try:
    from google.colab import files
    files.download('vectors.tsv')
    files.download('metadata.tsv')
except Exception as e:
    pass